# Unit 2.4 Hacks
> Creating my own database connecting to my CPT project
- toc: true
- comments: true
- categories: [csp, jupyter]
- permalink: /csp/24hackstocpt
- tags: [python]
- comments: true

In [ ]:
from flask import Flask
from flask_sqlalchemy import SQLAlchemy

"""
These object and definitions are used throughout the Jupyter Notebook.
"""

# Setup of key Flask object (app)
app = Flask(__name__)
# Setup SQLAlchemy object and properties for the database (db)
database = 'sqlite:///sqlite.db'  # path and filename of database
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False
app.config['SQLALCHEMY_DATABASE_URI'] = database
app.config['SECRET_KEY'] = 'SECRET_KEY'
db = SQLAlchemy()


# This belongs in place where it runs once per project
db.init_app(app)

In [ ]:
import datetime
from datetime import datetime
import json

from sqlalchemy.exc import IntegrityError
from werkzeug.security import generate_password_hash, check_password_hash




class User(db.Model):
    __tablename__ = 'users'  # table name is plural, class name is singular

    id = db.Column(db.Integer, primary_key=True)
    _task = db.Column(db.String(255), unique=False, nullable=False)
    _timeExpected = db.Column(db.String(255), unique=True, nullable=False)

    
    def __init__(self, task, timeExpected):

        self._task = task    # variables with self prefix become part of the object, 
        self._timeExpected = timeExpected

    @property
    def task(self):
        return self._task
    
    # a setter function, allows name to be updated after initial object creation
    @task.setter
    def name(self, task):
        self._task = task
    
    # a getter method, extracts uid from object
    @property
    def timeExpected(self):
        return self._timeExpected
    
    # a setter function, allows uid to be updated after initial object creation
    @timeExpected.setter
    def uid(self, timeExpected):
        self._timeExpected = timeExpected
        


    def __str__(self):
        return json.dumps(self.read())

    # CRUD create/add a new record to the table
    # returns self or None on error
    def create(self):
        try:
            # creates a person object from User(db.Model) class, passes initializers
            db.session.add(self)  # add prepares to persist person object to Users table
            db.session.commit()  # SqlAlchemy "unit of work pattern" requires a manual commit
            return self
        except IntegrityError:
            db.session.remove()
            return None

    # CRUD read converts self to dictionary
    # returns dictionary
    def read(self):
        return {
            "id": self.id,
            "task": self.task,
            "timeExpected": self.timeExpected,
        }

    # CRUD update: updates user name, password, phone
    # returns self
    def update(self, task="", timeExpected=""):
        """only updates values with length"""
        if len(task) > 0:
            self.task = task
        if len(timeExpected) > 0:
            self.uid = timeExpected
        db.session.commit()
        return self

    # CRUD delete: remove self
    # None
    def delete(self):
        db.session.delete(self)
        db.session.commit()
        return None
    # just a definition